### 3.1  apply与unapply
1. scala所有操作符都是左结合的, 即从左向右求值. 只有以下2种操作符是右结合的:  
  1. 赋值操作符  
  2. 以冒号(:)结尾的操作符  
  ```scala
  1::2::Nil 意思是 1::(2::Nil)
  ```

2. `apply`方法  
  1. apply方法用在伴生对象中, 负责实例化对象  
  2. 不用new实例化方法

In [8]:
class Fraction(n:Int,d:Int){}
object Fraction{
    def apply(n:Int,d:Int) = new Fraction(n,d)
}

new Fraction(2,3)

defined class Fraction
defined object Fraction
res7_2: wrapper.wrapper.Fraction = $sess.cmd7Wrapper$Helper$Fraction@2aa1bfb1

3. `unapply`方法   
 1. unapply是伴生对象中, 对apply方法的逆操作, 负责接收一个对象, 提取一个值     
 2. unapply提取的值返回Option类型/ Some类型

In [13]:
class Name( f:String, l:String){
    private def getF() = f
    private def getL() = l
}
object Name{
    // 提取器
    def unapply(obj:Name) = {
        Some((obj.getF(),obj.getL()))
    }
}
val Name(first,last) = new Name("liu","jie")

defined class Name
defined object Name
first: String = "liu"
last: String = "jie"

4. `unapply方法`, 有时只是测试其输入(例如模式匹配). 此时unapply返回Boolean

In [15]:
object IsCompound{
    def unapply(input:String) = input.contains(" ")
}
new Name("liu","jie ") match {
    case Name(first,last@IsCompound()) => print("one")
    case Name(first,last) => print("two")
}

one

defined object IsCompound

5. `unapplySeq`方法, 提取任意个数的值  
 `unapplySeq`返回`Option[Seq[]]`方法

In [16]:
object Name{
    def unapplySeq(in:String):Option[Seq[String]] = {
        if (in.trim=="") None else Some(in.trim.split(" "))
    }
}
"aa oo ss" match {
    case Name(f,l) => print(f+","+l)
    case Name(f,m,l) => print(f+","+m+","+l)
}

aa,oo,ss

defined object Name

### 3.2 高阶函数
#### 1. 函数是scala的一等公民, 因此可以将函数作为变量.  
#### 2. java中的函数概念不同于方法, 需要使用`function _`将方法转换为函数

In [25]:
import scala.math.ceil
// 将方法转换为函数
val fun = ceil _

import scala.math.ceil
// 将方法转换为函数

fun: Double => Double = <function1>

In [26]:
fun(1.1)

res25: Double = 2.0

#### 3. 可以传递匿名函数

In [27]:
Array(3,14,1.42,2.0).map(x=>3*x)

res26: Array[Double] = Array(9.0, 42.0, 4.26, 6.0)

#### 4. 函数作为参数

In [32]:
val fun = ceil _
// 函数作为参数
def value(f:(Double) => Double) = f(2.5)
println(value(fun))

3.0


fun: Double => Double = <function1>
defined function value

#### 5. 高阶函举例

In [33]:
(1 to 5).map("*" * _).foreach(println)

*
**
***
****
*****


In [34]:
(1 to 10).filter(_ %2 ==0)

res33: collection.immutable.IndexedSeq[Int] = Vector(2, 4, 6, 8, 10)

#### 6. SAM
1. SAM全称: 'single abstract method', 这是Java种的叫法.   
 因为java不能把函数作为参数传进另一个函数, 因此, Java想要实现这个功能, 就要把作为参数的函数抽象出一个接口来, 然后实例化这个接口的抽象方法产生对象作为参数传进去. 因为往往只实例化一次, 所以称为'单一抽象方法'.   
2. scala可以使用隐式转换将一个函数变成一个SAM对象. 

In [ ]:
implicit def mkAction(action: (ActionEvent)=>Unit) = {
    new ActionListener{
        override def actionPerformed(event:ActionEvent){
            // TODO
        }
    }
}

#### 7. 柯里化
1. 柯里化是将2个参数的函数, 变成'接收1个参数, 返回一个函数'的函数  
2. 且第二个函数可以利用第一个函数的参数.   
 如下例, mal2是mal1的柯里化形式

In [5]:
def mal1(x:Int) = (y:Int)=>x*y
def mal(x:Int)(y:Int) = x*y
mal1(6)

defined function mal1
defined function mal
res4_2: Int => Int = <function1>

3. `Seqlike`类中的`corresponds`方法利用了柯里化: 比较两个集合在某个对比条件下是否相同

```scala
  def corresponds[B](that: GenSeq[B])(p: (A,B) => Boolean): Boolean = {
    val i = this.iterator
    val j = that.iterator
    while (i.hasNext && j.hasNext)
      if (!p(i.next(), j.next()))
        return false

    !i.hasNext && !j.hasNext
  }
```

#### 8. 控制抽象  
有时候, 想把一块代码片段作为函数的参数使用, 此时可以把代码块表示为'不带参数,且返回值为unit类型的匿名函数'作为参数


In [36]:
def runThread(block: =>Unit) = {
    new Thread{
        override def run(){
            block // 执行代码片段
        }
    }.start()
}
// repel模式下, 代码块运行正常
runThread({println("hi");Thread.sleep(1000);println("bye")})

defined function runThread

#### 9. 使用柯里化和控制抽象, 实现条件转移代码块   

以下例子, 

In [6]:
// condition: 无参数,返回Boolean的函数
// block: 无参数, 返回Unit的函数
def until(condition: =>Boolean)(block: =>Unit):Unit = {
    if(!condition){
        block
        until(condition)(block)
    }
}

var x = 10
until(x==0){
    x = x-1
    println(x)
}

9
8
7
6
5
4
3
2
1
0


defined function until
x: Int = 0